##### This first cell will read an Excel file (single book) and flat their multiple hierarchical columns and the first column that contains space indentation in each row. Then it will create embeddings for each title rows and save it to a file.

In [ ]:
import pandas as pd
import json
#import openai
from openai import OpenAI
import numpy as np
from collections import deque
import os

"""
    Yield successive n-sized chunks from a list.

    Parameters:
    - lst (list): The input list to be chunked.
    - n (int): The size of each chunk.

    Yields:
    - chunk (list): A chunk of size n from the input list.

    This function iterates over the input list and yields successive chunks of size n.
    If the length of the list is not divisible evenly by n, the last chunk may be smaller
    than n. This function is useful for dividing a large list into smaller, more manageable
    chunks for processing or analysis.

    Example:
    >>> lst = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    >>> chunks = chunk_list(lst, 3)
    >>> for chunk in chunks:
    ...     print(chunk)
    [1, 2, 3]
    [4, 5, 6]
    [7, 8, 9]
    [10]
    """
def chunk_list(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

def get_embedding(text, model="text-embedding-ada-002"):
    text = text.replace("\n", " ").replace(".", "").replace("[", "").replace("]", "").replace("{", "").replace("}", "")
    res = client.embeddings.create(input = [text], model=model)
    emb = res.data[0].embedding
    return emb

def cosine_similarity(A, B):
    num = np.dot(A,B)
    sum1 = sum(map(lambda x: x * x, A))
    sum2 = sum(map(lambda x: x * x, B))
    den = np.sqrt(sum1) * np.sqrt(sum2)
    if(den == 0.0):
        return 0.0
    else:
        return float(num) / den

def load_environment_variables(file_path):
    with open(file_path, 'r') as file:
        settings = json.load(file)
        for key, value in settings.items():
            os.environ[key] = value


 """
    Process hierarchical data and generate a list of rows with hierarchical structure.

    Parameters:
    - data (list): A list containing strings representing hierarchical data.

    Returns:
    - rows (list): A list of strings representing rows with hierarchical structure.

    This function takes a list of strings representing hierarchical data where indentation
    indicates the hierarchy levels. It processes the data and generates a list of rows with
    hierarchical structure, where each row represents a level in the hierarchy. The rows
    are constructed by concatenating the hierarchical elements separated by dashes (-).

    Example:
    >>> data = [
    ...     'Áreas Gerais, Áreas Detalhadas e Tipo de Cursos ',
    ...     '  Ciência da educação',
    ...     '    Processos escolares',
    ...     '    Psicopedagogia',
    ...     '  Formação de professores de educação infantil',
    ...     '    Educação infantil formação de professor'
    ... ]
    >>> result = q_process_hierarchy(data)
    >>> print(result)
    ['Áreas Gerais, Áreas Detalhadas e Tipo de Cursos ',
     'Áreas Gerais, Áreas Detalhadas e Tipo de Cursos -Ciência da educação',
     'Áreas Gerais, Áreas Detalhadas e Tipo de Cursos -Ciência da educação-Processos escolares',
     'Áreas Gerais, Áreas Detalhadas e Tipo de Cursos -Ciência da educação-Psicopedagogia',
     'Áreas Gerais, Áreas Detalhadas e Tipo de Cursos -Formação de professores de educação infantil',
     'Áreas Gerais, Áreas Detalhadas e Tipo de Cursos -Formação de professores de educação infantil-Educação infantil formação de professor']
    """
def q_process_hierarchy(data):
    l = len(data)

    rows = []
    queue = deque()
    for item in data:
        if pd.isna(item):
            continue
        indent_level = len(item) - len(item.lstrip())
        while queue and queue[-1][1] >= indent_level:
            queue.pop()
        queue.append((item.strip(), indent_level))
        rows.append("-".join([row[0] for row in queue]))
        #print("-".join([row[0] for row in queue]))
    return rows

local_settings_path = 'local.settings.json'
load_environment_variables(local_settings_path)

op_key = os.environ["API_KEY"]
pddf = pd.read_excel('data/Libro2.xlsx', index_col=None, header=[0, 1, 2, 3])
pddf.columns = pddf.columns.to_flat_index()
# Remove substrings containing 'Unnamed: 0_level_'
pddf.columns = [(s[0],) + tuple(x for x in s[1:] if 'Unnamed:' not in x and 'Unnamed: 1_level_' not in x) for s in pddf.columns]

c1_nam = pddf.columns[0]
pddf = pddf.dropna(how='all')

expanded_df = q_process_hierarchy(pddf.iloc[:, 0])

l = len(expanded_df)

pddf.iloc[:, 0] = expanded_df

# emb_text = [chunk.get("('Áreas Gerais, Áreas Detalhadas e Tipo de Cursos ',)", "").strip() for chunk in list_of_dicts] 
# df_mod["emb_text"] = emb_text

# Add the 'Id' column at the beginning
#df_mod .insert(0, 'Id', range(1, len(df) + 1))


pddf.to_excel('data/Libro2Flat.xlsx')
pddf.to_csv('data/cLibro2Flat.csv', encoding='iso-8859-1', index=False)

df = pd.read_csv("data/cLibro2Flat.csv", encoding='latin-1')
json_per_row = df.to_json(indent=2, orient='records', force_ascii=False)

# Define the chunk size
chunk_size = 1
list_of_dicts = json.loads(json_per_row)

chunks = list(chunk_list(list_of_dicts, chunk_size))

# Convert list of dictionaries to JSON strings
json_chunks = [json.dumps(chunk, ensure_ascii=False) for chunk in chunks]

# Create DataFrame from JSON chunks
df_mod = pd.DataFrame({'data_text': json_chunks})

#create text for embeddings  ('Áreas Gerais, Áreas Detalhadas e Tipo de Cursos ',)
emb_text = [chunk.get("('Áreas Gerais, Áreas Detalhadas e Tipo de Cursos ',)", "").strip() for chunk in list_of_dicts] 

#emb_text = [chunk.get("Áreas Gerais, ", "").strip() + " "  + chunk.get("Áreas Detalhadas", "").strip() + " " + chunk.get("Tipo de Cursos", "").strip() + " "  for chunk in list_of_dicts] #+ chunk.get("Áreas Detalhadas", "")
df_mod["emb_text"] = emb_text

# Rename the column to 'data_chunks'
#df = df.rename(columns={0: 'data_chunks'})



# Add the 'embedding' column with empty values
#df['embedding'] = 0
embedding_model = "text-embedding-3-small" #"text-embedding-ada-002"
api_key = op_key
client = OpenAI(api_key=api_key)
#df_mod ['embeddings'] = df_mod.data_text.apply(lambda x: get_embedding(x, embedding_model))
df_mod ['embeddings'] = df_mod.emb_text.apply(lambda x: get_embedding(x, embedding_model))
df_mod.to_csv('data/Libro2FlatWithEmb.csv', encoding='iso-8859-1', index=False)

##### This second cell will read the file with the embeddings produced in the previous cell and will find those similar (or relevant to answer) a user question. Then it saves the results into a file.

In [ ]:
import pandas as pd
import json
#import openai
from openai import OpenAI
import numpy as np
import ast
import os


def chunk_list(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

def get_embedding(text, model="text-embedding-ada-002"):
    text = text.replace("\n", " ").replace(".", "").replace("[", "").replace("]", "").replace("{", "").replace("}", "")
    # text = text.replace("{", " ")
    # text = text.replace("}", " ")
    res = client.embeddings.create(input = [text], model=model)
    emb = res.data[0].embedding
    #emb = res['data'][0]['embedding']
    return emb

def cosine_similarity(A, B):
    num = np.dot(A,B)
    sum1 = sum(map(lambda x: x * x, A))
    sum2 = sum(map(lambda x: x * x, B))
    den = np.sqrt(sum1) * np.sqrt(sum2)
    if(den == 0.0):
        return 0.0
    else:
        return float(num) / den


local_settings_path = 'local.settings.json'
load_environment_variables(local_settings_path)
op_key = os.environ["API_KEY"]

embedding_model = "text-embedding-3-small" #"text-embedding-ada-002"
api_key = op_key
client = OpenAI(api_key=api_key)

df_mod = pd.read_csv("data/Libro2FlatWithEmb.csv", encoding='latin-1')

#query = "Tell me the number for Privada  Letras línguas estrangeiras clássicas formação de professor  "
query = "tell me the overal total number"
#query = "tell me the Educação total number"
#query = "tell me the average number for  Formação de professores de letras"


query_embedding = get_embedding(query, embedding_model)  #Find embeddings directly from the query

# Obtain the similarity of each fact to the query
#ada_embedding = df.embeddings.apply(eval).apply(np.array)

ada_embedding = df_mod ['embeddings']
i = 0 
for row_emb in ada_embedding:
    # Parse the string into a list of floats
    doubles_list = ast.literal_eval(row_emb)
    nprow_emb = np.array(doubles_list)
    num = cosine_similarity(query_embedding, nprow_emb)
    df_mod.loc[i,'similarity'] = 1 / (2 - num)  #(1 / 1 + distance)  monotonically decreasing
    i = i + 1

print(df_mod['similarity'])

similarity_array = df_mod['similarity'].values
quantiles = np.percentile(similarity_array, [25, 50, 75])

num = quantiles[2]

selected_rows = df_mod[df_mod['similarity'] >= num]

sel_row_emb_text = selected_rows['emb_text']
sel_row_emb_text.to_csv('data/sel_row_emb_text.csv')

print(selected_rows['emb_text'])

# Write the DataFrame to a CSV file
df_mod.to_csv('data/chunk_data_and_embeddings.csv', encoding='iso-8859-1', index=False)

